### RAG란?
- 데이터 소스로부터 Prompt 또는 질문과 연관된 데이터를 가져오는 행위
- Retrieval Augmented + Generation
  - Generation = ChatGPT
  - Retrieval = 연관 텍스트 가져오는 행위 (검색도 Retrieval의 일종)

### 왜 RAG가 필요할까?
- ChatGPT = 대량의 데이터를 학습한 대규모 언어 모델
- 학습 이후에 새로 나온 데이터 또는 학습하지 않은 데이터들에 대해서는 답변이 어려움
    - Knowledge Cutoff 또는 Training Data Cutoff
- 즉, RAG는 실시간으로 부족한 데이터를 주입하기 위해 고안된 방법

![GPT-4o Cutoff](./res/knowledge_cutoff.png)

In [2]:
import os

from openai import OpenAI


OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)

prompt = '내일 날씨는 어때?'

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

죄송합니다. 날씨 정보는 실시간으로 업데이트되는 정보이기 때문에 정확한 내일 날씨를 알려드릴 수 없습니다. 근처 기상청 웹사이트나 기상 앱을 통해 내일 날씨를 확인해보시는 것을 추천드립니다.


### 어떻게 실시간으로 부족한 데이터를 주입할까?
- Prompt에 부족한 데이터를 추가하는 방식

In [4]:
# 기상청 단기예보
weather_info = """(종합) 오늘 낮까지 비 곳, 오늘 강풍과 풍랑 유의, 내일 늦은 오후~저녁 경기동부 소나기, 모레 낮부터 비
○ (오늘, 30일) 대체로 흐림, 서울.경기도(경기남서부 제외) 중심 낮(12~15시)까지 비 곳
○ (내일, 7월 1일) 구름많음, 대기불안정으로 늦은 오후(15~18시)부터 저녁(18~21시) 사이 경기동부 소나기 곳, 서해5도 대체로 맑다가 저녁부터 구름많아짐
○ (모레, 7월 2일) 대체로 흐림, 서울.인천.경기도 낮(12~15시)부터 비, 서해5도(연평도 부근) 늦은 밤(21~24시) 가끔 비 곳
* 예상 강수량(30일 낮까지)
- 서울.경기도(경기남서부 제외): 5mm 미만
* 소나기에 의한 예상 강수량(7월 1일 늦은 오후~저녁)
- 경기동부: 5~20mm
* 예상 강수량(7월 2일 낮~)
- 서울.인천.경기도: 10~50mm
- 서해5도(7월 2일 밤): 5~20mm"""

prompt = '내일 날씨는 어때?' + '\n\n' + weather_info

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

내일은 구름많고 대기불안정으로 늦은 오후부터 저녁 사이에 경기동부에서 소나기가 올 것으로 예상됩니다. 모레는 대체로 흐리고 서울, 인천, 경기도에서 낮부터 비가 오겠습니다. 서해5도에서는 늦은 밤에 가끔 비가 올 것으로 예상됩니다. 오늘은 대체로 흐리고 서울, 경기도 중심으로 낮까지 비가 오는 곳이 있을 것으로 예상됩니다. 강풍과 풍랑에 유의하시기 바랍니다.


### 어떻게 질문과 연관된 정보를 가져올까? (RAG의 핵심)
- RAG에서 Retrieval이 나오는 이유
    - Retrieval = 연관 텍스트 가져오는 행위 (검색도 Retrieval에 포함)
- 어떻게 Retrieve 할까?
    - 방법 1) 텍스트에서 특정 단어가 자주 발생 --> 해당 단어랑 연관이 있을꺼다!
    - 방법 2) 근데 자주 발생하는 단어가 항상 자주 발생하는 단어 --> 이런건 가중치를 줄이자
    - 방법 3) 딥러닝 모델한테 텍스트 간의 문서를 직접 학습시키면 어떨까?
        - 2012년 Word2Vec 논문에서 시작. 흔히 임베딩이라고 한다.
- 가장 간단한 방법 1 시연

In [5]:
def count_common_words(text1, text2):
    words1 = text1.split()
    words2 = text2.split()
    common_words = set(words1) & set(words2)
    return len(common_words)


question = '한국의 전통 음식에 대해 무엇을 알고 계신가요?'

ref_texts = [
    '한국의 전통 음식은 다양한 재료와 조리법으로 유명합니다. 김치, 불고기, 비빔밥 등이 대표적인 예시입니다.',
    '한국의 음식 문화는 건강에 좋은 재료를 사용하는 것으로 알려져 있습니다. 발효 식품인 김치가 대표적입니다.',
    '한국의 역사와 문화는 매우 흥미롭습니다. 고궁과 한복, 그리고 태권도 등이 유명한 문화 요소입니다.'
]

common_word_counts = [count_common_words(question, ref_text) for ref_text in ref_texts]

most_similar_index = common_word_counts.index(max(common_word_counts))
print(f'가장 유사한 텍스트 번호: {most_similar_index + 1}')
print(f'겹치는 단어의 수: {common_word_counts[most_similar_index]}')

가장 유사한 텍스트 번호: 1
겹치는 단어의 수: 2


단어 기반 방법론의 문제점
- 단순하게 단어 빈도 기반이라 실제 맥락과 상관 없음 (=성능이 상대적으로 낮은 편)

### RAG의 동작 구조 정리
1. 사용자가 질문을 한다. (Prompt 입력)
2. RAG 로직을 통해 질문과 연관된 데이터를 가져와 Prompt에 추가한다.
3. ChatGPT는 추가된 데이터를 포함한 새로운 Prompt에 대해 답변을 한다.


### ChatGPT가 모르는 지식을 채우는 방법... RAG가 최선일까? (다른 방법론들)
- 크게 보면 LLM 학습 단계에서 주입해주거나 추론(답변)하는 단계에서 넣어 줄 수 있음
    - 사전훈련 (Pretraining)
    - 파인튜닝 (Finetuning)
    - RAG
    - Prompt Engineering - 외부 지식이 필요하므로 Prompt Engineering은 아웃
      - 정답은 없음. 실시간성이 높을 수록 RAG 빛나고, 그러지 않은 경우 이전 단계에서 1회 학습으로도 해결이 가능 할 수 있음


![OpenAI RAG Comparison](./res/rag_comparison.png)

출처: OpenAI DevDay

### RAG vs. Finetuning
- RAG의 장단점
  - 추가적인 LLM 학습이 필요 없고, 추론(답변) 단계에서 Prompt 추가로 바로 답변 가능
  - Input Token 수가 길어져 비용 증가로 이어짐
- Finetuning의 장단점
  - LLM 학습을 통해 외부 지식을 학습 할 수 있음
  - 외부 지식 외에도 태스크를 어떻게 잘 풀 수 있을 지도 추가 학습시킬 수 있음
  - 어쨌든 추가 학습 비용이 발생하며, 데이터가 많거나 모델이 클 경우 GPU 많이 필요 할 수 있음
- 위 2가지를 합친 방법론들도 나오고 있음 (ex. Retrieval Augmented Finetuning)

### 정리
- RAG란 LLM이 알고 있지 않은 지식을 실시간 주입을 하기 위한 방법론
- Retrieval, 즉 어딘가에서 정보를 가져오는데 구체적으로 어떻게 가져오는 지는 방법론이 여러가지 존재
- LLM 학습이 필요하진 않은 상대적으로 저렴한 방법론